In [1]:
import torch

# 檢查 CUDA 是否可用
cuda_available = torch.cuda.is_available()
print(f"CUDA 可用性: {cuda_available}")

# 如果 CUDA 可用，顯示 CUDA 設備數量
if cuda_available:
    print(f"可用的 CUDA 設備數量: {torch.cuda.device_count()}")
    print(f"當前使用的 CUDA 設備: {torch.cuda.current_device()}")
    print(f"CUDA 設備名稱: {torch.cuda.get_device_name(torch.cuda.current_device())}")


CUDA 可用性: True
可用的 CUDA 設備數量: 1
當前使用的 CUDA 設備: 0
CUDA 設備名稱: NVIDIA GeForce GTX 1650


In [4]:
import pyzed.sl as sl
import cv2

# 創建ZED對象
zed = sl.Camera()

# 配置初始化參數
init_params = sl.InitParameters()
init_params.camera_resolution = sl.RESOLUTION.HD720
init_params.depth_mode = sl.DEPTH_MODE.PERFORMANCE
init_params.coordinate_units = sl.UNIT.METER

# 打開ZED相機
err = zed.open(init_params)
if err != sl.ERROR_CODE.SUCCESS:
    exit(1)

runtime_parameters = sl.RuntimeParameters()

# 創建影像對象
image_zed = sl.Mat()
depth_zed = sl.Mat()

while True:
    if zed.grab(runtime_parameters) == sl.ERROR_CODE.SUCCESS:
        # 獲取RGB影像
        zed.retrieve_image(image_zed, sl.VIEW.LEFT)
        # 獲取深度影像
        zed.retrieve_measure(depth_zed, sl.MEASURE.DEPTH)
        
        # 將ZED影像轉換為OpenCV格式
        image_ocv = image_zed.get_data()
        depth_ocv = depth_zed.get_data()

        # 顯示影像
        cv2.imshow("ZED Image", image_ocv)
        cv2.imshow("ZED Depth", depth_ocv)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

zed.close()
cv2.destroyAllWindows()


## 物件偵測

In [7]:
import sys
import pyzed.sl as sl
import torch
import numpy as np
import cv2
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import non_max_suppression, scale_coords

# 初始化 ZED 相機
zed = sl.Camera()
init_params = sl.InitParameters()
init_params.camera_resolution = sl.RESOLUTION.HD720
init_params.coordinate_units = sl.UNIT.METER

if zed.open(init_params) != sl.ERROR_CODE.SUCCESS:
    print("無法開啟 ZED 相機")
    exit(1)

runtime_params = sl.RuntimeParameters()
runtime_params.sensing_mode = sl.SENSING_MODE.STANDARD

# 初始化 YOLOv5 模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DetectMultiBackend('yolov5m.pt', device=device)
model.eval()

# 配置攝影機影像大小
image_size = 640
conf_threshold = 0.3

def detect_objects(image):
    """對影像進行物件偵測，並返回結果。"""
    img = cv2.resize(image, (image_size, image_size))
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device).float() / 255.0

    with torch.no_grad():
        pred = model(img)
    results = non_max_suppression(pred, conf_threshold)

    return results[0]  # 返回偵測結果

def draw_boxes(image, detections, depth):
    """在影像上繪製偵測框及顯示 3D 座標。"""
    h, w, _ = image.shape

    for det in detections:
        if det is None:  
            continue
        x1, y1, x2, y2, conf, cls = map(int, det[:6].tolist())
        label = f"{model.names[cls]} {conf:.2f}"

        # 取得物件中心點的深度資訊
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        depth_value = depth.get_value(cx, cy)[1]

        # 繪製邊界框與標籤
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"{label} Depth: {depth_value:.2f}m", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

while True:
    image = sl.Mat()
    depth = sl.Mat()

    # 擷取 ZED 影像與深度
    if zed.grab(runtime_params) == sl.ERROR_CODE.SUCCESS:
        zed.retrieve_image(image, sl.VIEW.LEFT)
        zed.retrieve_measure(depth, sl.MEASURE.DEPTH)

        img_np = image.get_data()
        detections = detect_objects(img_np)

        draw_boxes(img_np, detections, depth)

        # 顯示影像
        cv2.imshow("ZED YOLOv5 Detection", img_np)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# 釋放資源
cv2.destroyAllWindows()
zed.close()


ImportError: cannot import name 'scale_coords' from 'yolov5.utils.general' (d:\python\zed\yolov5\utils\general.py)